In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/samsum-dataset-text-summarization/samsum-train.csv
/kaggle/input/samsum-dataset-text-summarization/samsum-test.csv
/kaggle/input/samsum-dataset-text-summarization/samsum-validation.csv
/kaggle/input/samsum-dataset-text-summarization/samsum_dataset/dataset_dict.json
/kaggle/input/samsum-dataset-text-summarization/samsum_dataset/validation/state.json
/kaggle/input/samsum-dataset-text-summarization/samsum_dataset/validation/dataset_info.json
/kaggle/input/samsum-dataset-text-summarization/samsum_dataset/validation/data-00000-of-00001.arrow
/kaggle/input/samsum-dataset-text-summarization/samsum_dataset/test/state.json
/kaggle/input/samsum-dataset-text-summarization/samsum_dataset/test/dataset_info.json
/kaggle/input/samsum-dataset-text-summarization/samsum_dataset/test/data-00000-of-00001.arrow
/kaggle/input/samsum-dataset-text-summarization/samsum_dataset/train/state.json
/kaggle/input/samsum-dataset-text-summarization/samsum_dataset/train/dataset_info.json
/kaggle/input/sam

In [2]:
train_data = pd.read_csv("/kaggle/input/samsum-dataset-text-summarization/samsum-train.csv")
train_data.drop('id',axis=1,inplace=True)
train_data.head(5)

,dialogue,summary
0,Amanda: I baked cookies. Do you want some?\r\...,Amanda baked cookies and will bring Jerry some...
1,Olivia: Who are you voting for in this electio...,Olivia and Olivier are voting for liberals in ...
2,"Tim: Hi, what's up?\r\nKim: Bad mood tbh, I wa...",Kim may try the pomodoro technique recommended...
3,"Edward: Rachel, I think I'm in ove with Bella....",Edward thinks he is in love with Bella. Rachel...
4,Sam: hey overheard rick say something\r\nSam:...,"Sam is confused, because he overheard Rick com..."


In [3]:
test_data = pd.read_csv("/kaggle/input/samsum-dataset-text-summarization/samsum-test.csv")
test_data.drop('id',axis=1,inplace=True)
test_data.head(5)

,dialogue,summary
0,"Hannah: Hey, do you have Betty's number?\nAman...",Hannah needs Betty's number but Amanda doesn't...
1,Eric: MACHINE!\r\nRob: That's so gr8!\r\nEric:...,Eric and Rob are going to watch a stand-up on ...
2,"Lenny: Babe, can you help me with something?\r...",Lenny can't decide which trousers to buy. Bob ...
3,"Will: hey babe, what do you want for dinner to...",Emma will be home soon and she will let Will k...
4,"Ollie: Hi , are you in Warsaw\r\nJane: yes, ju...",Jane is in Warsaw. Ollie and Jane has a party....


In [4]:
val_data = pd.read_csv("/kaggle/input/samsum-dataset-text-summarization/samsum-validation.csv")
val_data.drop('id',axis=1,inplace=True)
val_data.head(5)

,dialogue,summary
0,"A: Hi Tom, are you busy tomorrow’s afternoon?\...",A will go to the animal shelter tomorrow to ge...
1,Emma: I’ve just fallen in love with this adven...,Emma and Rob love the advent calendar. Lauren ...
2,Jackie: Madison is pregnant\r\nJackie: but she...,Madison is pregnant but she doesn't want to ta...
3,Marla: <file_photo>\r\nMarla: look what I foun...,Marla found a pair of boxers under her bed.
4,Robert: Hey give me the address of this music ...,Robert wants Fred to send him the address of t...


In [5]:
print(f"The shape of train_data is {train_data.shape}")
print(f"The shape of train_data is {test_data.shape}")
print(f"The shape of train_data is {val_data.shape}")

The shape of train_data is (14732, 2)
The shape of train_data is (819, 2)
The shape of train_data is (818, 2)


In [6]:
%%capture
!pip install transformers[torch]
!pip install rouge_score

In [7]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq
import numpy as np

In [8]:
checkpoint = 't5-small'
tokenizer = T5Tokenizer.from_pretrained(checkpoint,legacy=False)

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [9]:
def preprocess_function(examples):
    
    inputs = ["summarize: " + (doc if doc is not None else "") for doc in examples["dialogue"]]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["summary"], max_length=150, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


In [10]:
from datasets import Dataset

train_data = Dataset.from_pandas(train_data)
val_data = Dataset.from_pandas(val_data)
test_data = Dataset.from_pandas(test_data)

tokenized_train = train_data.map(preprocess_function, batched=True)
tokenized_validation = val_data.map(preprocess_function, batched=True)
tokenized_test = test_data.map(preprocess_function, batched=True)

Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:4117: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/818 [00:00<?, ? examples/s]

Map:   0%|          | 0/819 [00:00<?, ? examples/s]

In [11]:
model = T5ForConditionalGeneration.from_pretrained(checkpoint)

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [12]:
tokenized_train

Dataset({
    features: ['dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 14732
})

In [13]:
!pip install evaluate

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.6 MB/s eta 0:00:00


In [14]:
!pip install evaluate

In [15]:
from transformers import T5Tokenizer
import numpy as np
import evaluate

tokenizer = T5Tokenizer.from_pretrained('t5-small')
rouge = evaluate.load("rouge")

def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions
    pad_token_id = tokenizer.pad_token_id
    labels_ids[labels_ids == -100] = pad_token_id
    pred_ids = np.clip(pred_ids, 0, tokenizer.vocab_size - 1).astype(int)
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)
    rouge_output = rouge.compute(predictions=pred_str, references=label_str, use_stemmer=True)
    result = {key: value * 100 for key, value in rouge_output.items()}
    prediction_lens = [np.count_nonzero(pred != pad_token_id) for pred in pred_ids]
    result["gen_len"] = np.mean(prediction_lens)

    return result

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [16]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",             
    eval_strategy="epoch",        
    learning_rate=2e-5,                 
    per_device_train_batch_size=16,     
    per_device_eval_batch_size=16,      
    weight_decay=0.01,                  
    save_total_limit=3,                 
    num_train_epochs=10,                 
    predict_with_generate=True,         
    generation_max_length=150,          
    generation_num_beams=4,            
    run_name="unique_run_name"
)

In [17]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

trainer = Seq2SeqTrainer(
    model=model,                       
    args=training_args,              
    train_dataset=tokenized_train,     
    eval_dataset=tokenized_validation, 
    data_collator=data_collator,       
    tokenizer=tokenizer,               
    compute_metrics=compute_metrics,   
)

In [18]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [19]:
trainer.train()

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Tracking run with wandb version 0.18.3
wandb: W&B syncing is set to `offline` in this directory.  
wandb: Run `wandb online` or set WANDB_MODE=online to enable cloud syncing.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,2.264300,1.871783,42.983830,19.651775,35.419746,35.371736,25.872861
2,2.042200,1.821443,44.385392,20.845114,36.704068,36.628021,26.150367
3,1.983000,1.790728,45.207587,21.345138,37.276276,37.213588,27.566015
4,1.943400,1.776358,45.671480,21.590496,37.517288,37.457155,28.537897
5,1.919000,1.759781,45.932457,22.172746,38.123696,38.047422,26.418093
6,1.892900,1.751128,46.401305,22.435450,38.463348,38.427313,27.721271
7,1.886700,1.747234,46.223003,22.243074,38.316995,38.269448,27.610024
8,1.874400,1.742875,46.410079,22.546935,38.402485,38.365012,27.354523
9,1.874100,1.740121,46.411590,22.669187,38.560664,38.520357,27.281174
10,1.862500,1.738513,46.572432,22.626737,38.631951,38.592610,27.410758


TrainOutput(global_step=9210, training_loss=1.9464632181339492, metrics={'train_runtime': 3291.323, 'train_samples_per_second': 44.76, 'train_steps_per_second': 2.798, 'total_flos': 1.5081510185336832e+16, 'train_loss': 1.9464632181339492, 'epoch': 10.0})

In [20]:
trainer.evaluate()

test_results = trainer.evaluate(eval_dataset=tokenized_test)

print(test_results)

{'eval_loss': 1.7491981983184814, 'eval_rouge1': 45.92495005113327, 'eval_rouge2': 21.582989338645007, 'eval_rougeL': 37.40956957311625, 'eval_rougeLsum': 37.40106115570963, 'eval_gen_len': 27.965811965811966, 'eval_runtime': 94.7232, 'eval_samples_per_second': 8.646, 'eval_steps_per_second': 0.549, 'epoch': 10.0}


In [21]:
import torch
from torch.utils.data import Dataset

# Define a custom dataset class for your random texts
class CustomTextDataset(Dataset):
    def __init__(self, texts, tokenizer):
        self.texts = texts
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        # Tokenize the text and return input_ids and attention_mask
        encoding = self.tokenizer(self.texts[idx], padding='max_length', truncation=True, return_tensors="pt", max_length=512)
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten()
        }

# Define your random text input
random_texts = [
    """Some of the best AI chatbots for coding include:
GitHub Copilot
Helps developers write high-quality code more efficiently. It can suggest lines of code, complete functions, write comments, and help with debugging. 
Amazon CodeWhisperer
Provides real-time code recommendations to improve coding efficiency. It offers features like code suggestions, function completion, documentation generation, and security scanning. 
ChatGPT
A generalized AI chatbot that can generate code samples in many languages. It can also provide detailed explanations and engage in substantive conversations. 
Claude 3.5 Sonnet
Anthropic's Claude AI chatbot is considered ideal for programming. It can handle uploaded files, process more words than the free version of ChatGPT, and access websites. 
Codacy
Uses automated tools to improve code quality and security. It also has code review features to promote collaboration and learning. 
Codeium
A free solution that includes an AI chatbot, intelligent autocomplete, and context-aware code generation. 
Other AI coding assistants include: Tabnine and MutableAI."""
    
]

# Create a dataset from the random texts
test_dataset = CustomTextDataset(random_texts, tokenizer)

# Use the trainer to predict
predictions = trainer.predict(test_dataset)

# Decode predictions to readable text
decoded_predictions = tokenizer.batch_decode(predictions.predictions, skip_special_tokens=True)

# Print the generated outputs
for i, text in enumerate(random_texts):
    print(f"Input: {text}\nGenerated Output: {decoded_predictions[i]}\n")

Input: Some of the best AI chatbots for coding include:
GitHub Copilot
Helps developers write high-quality code more efficiently. It can suggest lines of code, complete functions, write comments, and help with debugging. 
Amazon CodeWhisperer
Provides real-time code recommendations to improve coding efficiency. It offers features like code suggestions, function completion, documentation generation, and security scanning. 
ChatGPT
A generalized AI chatbot that can generate code samples in many languages. It can also provide detailed explanations and engage in substantive conversations. 
Claude 3.5 Sonnet
Anthropic's Claude AI chatbot is considered ideal for programming. It can handle uploaded files, process more words than the free version of ChatGPT, and access websites. 
Codacy
Uses automated tools to improve code quality and security. It also has code review features to promote collaboration and learning. 
Codeium
A free solution that includes an AI chatbot, intelligent autocomplete,

In [22]:
test_data

Dataset({
    features: ['dialogue', 'summary'],
    num_rows: 819
})